# Assistant API quickstart with a math tutor example and code interpreter

In [7]:
# from dotenv import load_dotenv
# load_dotenv()  # take environment variables from .env.
import os
from openai import OpenAI

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)


## Step 1 Create an assistant

In [8]:

assistant = client.beta.assistants.create(
  name="Math Tutor",
  instructions="You are a personal math tutor. Write and run code to answer math questions.",
  tools=[{"type": "code_interpreter"}],
  model="gpt-4o",
)

## Step 2 Create an thread

In [9]:
thread = client.beta.threads.create()

## Step 3 Add a Message to the Thread

In [16]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

## Step 4.1 Create a Run without streaming

In [17]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
messages
print(messages[0].content[0].text)

Text(annotations=[], value="Of course, Jane Doe! Let's solve the equation \\(3x + 11 = 14\\).\n\n1. Start with the equation:\n   \\[ 3x + 11 = 14 \\]\n\n2. Subtract 11 from both sides to isolate the term with \\(x\\):\n   \\[ 3x = 14 - 11 \\]\n\n3. Simplify the right side:\n   \\[ 3x = 3 \\]\n\n4. Divide both sides by 3 to solve for \\(x\\):\n   \\[ x = \\frac{3}{3} \\]\n\n5. Simplify the division:\n   \\[ x = 1 \\]\n\nSo, the solution is \\(x = 1\\). If there's anything more you'd like to ask, feel free!")


## Step 4.2 Create a Run with streaming

In [ ]:
from typing_extensions import override
from openai import AssistantEventHandler
 
# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.
 
class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)
 
# Then, we use the `stream` SDK helper 
# with the `EventHandler` class to create the Run 
# and stream the response.
 
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account.",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()

In [13]:
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
  print(messages[0].content[0].text)
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_vt8zAFGHtlXZxF9fCccAH9zu', assistant_id='asst_PbSspPumR6I23NBTxYeoEFCf', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Certainly, Jane Doe! The solution to the equation \\(3x + 11 = 14\\) is \\(x = 1\\).'), type='text')], created_at=1729153093, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_i3gcV7BOYJgoUzdz1Qk0MD9t', status=None, thread_id='thread_HoIJhYiRYfj3XrcMODrSbDFy'), Message(id='msg_A3ivHuvGcqcLrLqekYo4e9Te', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')], created_at=1729153083, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_HoIJhYiRYfj3XrcMODrSbDFy')], object='list', first_id='msg_vt8zA

TypeError: 'SyncCursorPage[Message]' object is not subscriptable